# Worksheet 08

Name: Yuzhe Jiang

UID: U92913042

Link to my repo: https://github.com/jiangyz112/Data-Science-Fundamentals/tree/worksheet_08

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [ ]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [ ]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [ ]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:  # Coin lands on H
        p1 = s1.pop()  # Remove and return the last item from s1
        data.append(p1)
    else:  # Coin lands on T
        p2 = s2.pop()  # Remove and return the last item from s2
        data.append(p2)
print(data)


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Component 1:
Mean (μ1): 5 
Variance (σ1^2): 1 
Mixture Weight (π1): Unknown 


Component 2:
Mean (μ2): 8
Variance (σ2^2): 1
Mixture Weight (π2): Unknown 


Number of Components (K): 2 
Total Number of Data Points (N): 20 

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [ ]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)


prob_s = [len(s1) / (len(s1) + len(s2)), len(s2) / (len(s1) + len(s2))]
mean_s = [np.mean(s1), np.mean(s2)]
var_s = [np.var(s1, ddof=0), np.var(s2, ddof=0)]

print(f"P(S_1) = {prob_s[0]},  P(S_2) = {prob_s[1]}")
print(f"mean_1 = {mean_s[0]},  mean_2 = {mean_s[1]}")
print(f"var_1 = {var_s[0]},  var_2 = {var_s[1]}")


P(S_1) = 0.4,  P(S_2) = 0.6, Which means that the first component accounts for 40% of the dataset and the second component accounts for 60%.

mean_1 = 4.772441743304363,  mean_2 = 8.26583890766299, They are quite close to the true value 5 and 8.

var_1 = 0.5648094262922441,  var_2 = 0.3541115604665342, These estimates indicate some degree of bias compared to the true variance1.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [ ]:
from scipy.stats import norm

prob_s0_x = []  # P(S_0 | X_i)
prob_s1_x = []  # P(S_1 | X_i)
prob_x = []  # P(X_i)

k = 2
mean_s = [4.686447898567738, 7.940330366281063]  # Updated mean values from K-means
var_s = [0.5023381614641249, 0.377297327876015]  # Updated variance values from K-means
prob_s = [0.5, 0.5]  # Mixture probabilities

for p in data:
    pdf_i = []
    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean_s[j], np.sqrt(var_s[j])))

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x_i = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]
    prob_x.append(prob_x_i)

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x_i)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x_i)

probs = zip(data, prob_s0_x, prob_s1_x)

for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

#8.13, 8.15, 8.35, 6.85, 7.46, 9.08 have a very high probability of belonging to the second cluster.
#4.19, 5.95, 6.36, 6.33 have a very high probability of belonging to the first cluster.
#Which is similar to the truth.

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [ ]:
# Updating the estimates for mean_j, var_j, and P(S_j) based on P(S_j | X_i)
prob_c = [sum(prob_s0_x) / len(prob_s0_x), sum(prob_s1_x) / len(prob_s1_x)]
mean_updated = [
    sum(p * x for p, x in zip(prob_s0_x, data)) / sum(prob_s0_x),
    sum(p * x for p, x in zip(prob_s1_x, data)) / sum(prob_s1_x)
]
var_updated = [
    sum(p * (x - mean_updated[0])**2 for p, x in zip(prob_s0_x, data)) / sum(prob_s0_x),
    sum(p * (x - mean_updated[1])**2 for p, x in zip(prob_s1_x, data)) / sum(prob_s1_x)
]

print(f"P(S_1) = {prob_s[0]},  P(S_2) = {prob_s[1]}")
print(f"mean_1 = {mean_s[0]},  mean_2 = {mean_s[1]}")
print(f"var_1 = {var_s[0]},  var_2 = {var_s[1]}")
print()

P(S_1) = 0.3999655099972517,  P(S_2) = 0.6000344900027483

mean_1 = 4.772889939258388,  mean_2 = 8.265339352710527

var_1 = 0.5665935358249932,  var_2 = 0.3557228640015676

They are all quiet close to the original ones. But they are all a little more accurate.


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [ ]:
# Recomputing P(S_j | X_i) based on the updated estimates of mean_j, var_j, and P(S_j)

prob_s0_x_updated = []  # P(S_0 | X_i) updated
prob_s1_x_updated = []  # P(S_1 | X_i) updated

for p in data:
    pdf_i_updated = [
        norm.pdf(p, mean_updated[0], np.sqrt(var_updated[0])),
        norm.pdf(p, mean_updated[1], np.sqrt(var_updated[1]))
    ]

    # Recompute P(X_i)
    prob_x_i_updated = prob_c[0] * pdf_i_updated[0] + prob_c[1] * pdf_i_updated[1]

    # Recompute P(S_j | X_i)
    prob_s0_x_updated.append(pdf_i_updated[0] * prob_c[0] / prob_x_i_updated)
    prob_s1_x_updated.append(pdf_i_updated[1] * prob_c[1] / prob_x_i_updated)

probs_updated = zip(data, prob_s0_x_updated, prob_s1_x_updated)

for p in probs_updated:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

The output are all very close to original one but fit data distribution more.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [ ]:
# Creating a hard assignment based on the updated P(S_j | X_i)
hard_assignments = [0 if prob_s0 > prob_s1 else 1 for prob_s0, prob_s1 in zip(prob_s0_x_updated, prob_s1_x_updated)]

print(hard_assignments)
